# Installing the ComAp package
1. You need to have `python` 3.6 or newer installed
2. Download the latest release from the `ComAp-API` github repository
3. Run `setup.bat`. It will install the comap API package and lead you to generate the API secrets (see further)

# Authentication - API secrets
To link the API to your account, and for security reasons, each API call needs to have two parameters: `ComAp-Key` and `Token`.  
To get the ComAp-Key, login to https://portal.websupervisor.net/developer (using your WSV Pro login credentials). There are two keys available - Primary and Secondary - pick one of them - it does not matter which one.

*(The idea is, that you use one of them in production, and the second for testig. After the testing, you can Regenerate the key, to make sure it stays secret. And this will not impact the other key that is used in production.)*

### Generating token
The `Token` is generated by calling API `Authenticate`. As part of this package, there is a script `get-token.py`, that will ask you for the `ComAp-Key`, your WSV username and password, generare the `Token` and store it in the config file. This script is called from the `setup.bat` script. But you can call it any time by running `python get-token` from the command line. 
Please do not share this configuration file, it contains your login information. 
The `Token` automatically expires after 1 month if it is not used. The `ComAp-Key` can be regenerated in case it gets leaked.

### Storing the API secrets 
The API key should not be hard-coded in the script. There are number of ways for storing the keys securely, that are beyond the scope of this document. For the purpose of this repository, we are storing the key in file `config.py`, with following content:
``` python
KEY = '<YOUR KEY GOES HERE>'
TOKEN = '<THE TOKEN GOES HERE>'
```
The individual scripts then get the stored secrets by calling:

In [1]:
from config import KEY, TOKEN 

# Calling WebSupervisor
All ComAp WebSupervisor APIs are methods of `wsv` class. To use them, you need to authenticate to an instance:

In [2]:
from comap.api import wsv

wsv = wsv(KEY, TOKEN)

Then you can use individual functions.

## Units
To get the list of units (gensets) under your WSV account, call method `wsv.units`.

It returns a list of units, each with the following attributes: `name`, `unitGuid`, `url`.

In [3]:
for unit in wsv.units():
    print(unit["name"])

IL3 AMF25 - nahradnik - GSM - test
Hybrid Barcelona
CGT Polyglass
IL-NT
ID FLX lite
Cogeneration unit ABB Prague
ID-Lite
IA-NT STD
IC-NT Mint
Cogeneration unit Hilton Longon
ID-EM
Harvester Goa
Habana hospital
IG-NT SPtM &#x28;CHP&#x29;
AIO GAS
New marine
IL9
Controller Parallel
Controller Solo
Hybrid
test33
sifrovani IL3
Real CHP IT
C07 - Hybrid Master
IG200_VAJL


## Info
This is to get the information about an unit. For this method (and all the following ones, we need to specify the unit by its `unitGuid`. You can get this information from calling units, or from the WebSupervisor web unit detail url.

For this example, let's use unitGuid `genset98a2e2828bde4ddb8d48fe5a39cade27`

In [4]:
unitGuid = 'genset98a2e2828bde4ddb8d48fe5a39cade27'
info = wsv.info(unitGuid)
values = [('Name', info['name']),
          ('Owner', info['ownerLoginId']),
          ('Timezone', info['timezone']),
          ('Position', f'{info["position"]["latitude"]},{info["position"]["longitude"]}')
]
for desc, value in values:
    print(f'{desc:10}{value}')

Name      ID-EM
Owner     storage
Timezone  Europe/Prague
Position  50.10697,14.45295


## Values
WIth this you can read controller values. You can read all values (not recommended) or list of specific values. To specify the values to get, you need to include comma separated list of their `valueGuid`s. 

The list of available value depends on the type of controller. You can get the list of available `valueGuid`s by calling it without any `valueGuid`s.

The list of common `valueGuid`s is available from constant `VALUE_GUID`. Let's check it:

In [5]:
from comap.constants import VALUE_GUID
for value in VALUE_GUID:
    print(f'{value:18} {VALUE_GUID[value]}')

genset_name        7b2ae258-65a8-40dd-bb42-5455753679f9
serial_number      6253525D-CF01-4a15-A34B-6C232496E7B4
last_update        0C9117DA-495A-11DF-85EB-428556D89593
comm_state         F0219C1C-1860-4b4d-8E6E-3CEC96279D6F
unit_state         48EC10D1-7AF8-4ce9-B5B3-A5315993FEB3
unit_changed       52C8C105-49E6-477B-B43D-33FD46F54640
controller         B9FF3CE7-0A81-4D80-8029-99C24A0E764F
alarm_list         7EDE9575-FEB5-4540-9153-A3B9CF60DE9F
alarm_list_ext     5E36C1AA-A2B7-4264-96B5-BB975E48B547
mode               6a12aed6-3be0-44c4-9110-9ea33cfe3ccc
nominal_power      72D0295A-3E65-11DF-892C-D6A856D89593
actual_power       EE83E7E9-7453-4e64-A5CB-C8C093FB2A2F
engine_state       BB2D1ADE-740E-488d-853B-6BA970D52E27
breaker_state      B7E5B9B1-9F5E-45c1-82F4-3A336F1F97FA
Gcb_mode           b0f910f3-14e4-4bd1-8db1-739e26680b9c
Mcb_mode           5bbc94e2-2497-4d2b-a3e6-1c798d0b6430
import_kWh         4C214D52-3D75-11DF-8AA2-529056D89593
U_L1               4a2a2694-3049-4a8e-ba5f-cc6fe

Now let's read few values:

In [6]:
values = wsv.values(
    unitGuid,
    f'{VALUE_GUID["mode"]}, '
    f'{VALUE_GUID["engine_state"]}'
)
for value in values:
    print(f'{value["name"]:<16} : {value["value"]}')

ControllerMode   : MAN


## History
To get log of the values for given time period (do not confuse with the controller history)

In [7]:
history = wsv.history(
    unitGuid,
    '09/19/2019',
    '09/20/2019',
    VALUE_GUID["engine_state"]
)
for event in history[0]["history"]:
    print(f'{event["value"]:<8} valid to {event["validTo"]}')

Ready    valid to 2019-09-19 08:52:44
Ready    valid to 2019-09-19 12:50:23
Ready    valid to 2019-09-21 06:29:23


## Files
List the files (reports and controller history) stored on the controller.

In [8]:
for file in wsv.files(unitGuid):
    print(f'{file["generated"]} {file["fileType"]} [{file["fileName"]}]')

2019-09-19 18:45:42 unitHistory [2019-09-19_20-44_ID-EM.csv]
2019-09-19 18:47:27 unitHistory [2019-09-19_20-45_ID-EM.csv]


## Comments
Get the comments entered on WSV Application. This can be used to automate the intereaction with the application users. 

In [9]:
for comment in wsv.comments(unitGuid):
    print(f'{comment["author"]:<18} {comment["date"]} {comment["text"]}')

Vaclav Chaloupka   2019-09-19 17:08:15 Ciao


## Download
This can be used to download a file

In [10]:
FILENAME = 'filename.csv'

if wsv.download(unitGuid,FILENAME):
    print('SUCCESS')
else:
    print('FAILED')

SUCCESS


# Asynchronous package
The previous examples are using synchronous Python package `requests`. Whilst simple for understanding, the disadvantage of this package is, that it blocks computer resources while waiting for the server response.

There is therefore also Asynchronous module available. It has the same methods as described above, but they start with a word `_async`.  Explaining the `aiohttp` package is beyond the scope of this document, if you intend using the async package, I assume you are familiar with the concept.

Here is an example of usage - doing the same as `wsv.units()`

In [ ]:
import aiohttp
import asyncio
from comap.api_async import wsv_async

async def test():
    session = aiohttp.ClientSession(raise_for_status=True)
    wsv = wsv_async(session, KEY, TOKEN)
    for unit in await wsv.async_units():
        print(f'{unit["name"]}')
    await session.close()

asyncio.run(test())